In [1]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

In [2]:
import pandas
import numpy
from matplotlib import pyplot as plt
import seaborn
%matplotlib notebook
import datetime
import general
import discretionary_aua
import vantage_aua
import data_accessing
import combined
import revenue
import costs
import consolidated
import discf
from ipywidgets import widgets
from IPython.display import display
import stats
from IPython.display import clear_output
pandas.set_option('display.float_format', '{:20,.6f}'.format)   #suppressing scientific notation
idx = pandas.IndexSlice

In [3]:
print('Pulling data...')
fund_dic = data_accessing.read_data(data_accessing.fund_data_name,data_accessing.fund_data_types)
nnb_dic = data_accessing.read_data(data_accessing.nnb_data_name,data_accessing.nnb_data_sheet)
discretionary_aua.append_share_class_units(fund_dic) # appending share class units
index_hl_dic = data_accessing.read_data(data_accessing.index_hl_data_name, data_accessing.index_hl_data_sheet)
data_dic = {**fund_dic, **index_hl_dic, **nnb_dic}

input_dic = data_accessing.read_data(data_accessing.assumptions_name, data_accessing.assumptions_sheet)

print('Data pulled successfully.')

Pulling data...
Data pulled successfully.


In [4]:
assumption_button = widgets.Button(description='Update assumptions',layout=widgets.Layout(width='20%',height='40px'))
assumption_button.style.button_color = 'lightgreen'
def update_assumption(b):
    global input_dic
    input_dic = data_accessing.read_data(data_accessing.assumptions_name, data_accessing.assumptions_sheet)
    print ('Assumptions updated successfully...')

assumption_button.on_click(update_assumption)
display(assumption_button)

Button(description='Update assumptions', layout=Layout(height='40px', width='20%'), style=ButtonStyle(button_color='lightgreen'))

In [5]:
revenue_types = ['Semi-annual revenue','Annual revenue - FY','Annual revenue - CY','Consolidated semi-annual revenue','Consolidated annual revenue - FY','Consolidated annual revenue - CY']
box_layout = widgets.Layout(display='flex', flex_flow='column', align_items='stretch',border='solid',width='30%')
items = {}
for w in revenue_types:
    items[w] = widgets.Button(description=w,layout=widgets.Layout(width='100%', height='40px'))
    items[w].style.button_color = 'lightgreen'
left_box = widgets.VBox([list(items.values())[0],list(items.values())[1],list(items.values())[2]],layout=box_layout)
right_box = widgets.VBox([list(items.values())[3],list(items.values())[4],list(items.values())[5]],layout=box_layout)
items['Semi-annual revenue'].on_click(lambda x: display(revenue.semi_revenue(data_dic,input_dic)))
items['Annual revenue - FY'].on_click(lambda x: display(revenue.annual_revenue(data_dic,input_dic)))
items['Annual revenue - CY'].on_click(lambda x: display(revenue.annual_revenue(data_dic,input_dic,cal_year=True)))
items['Consolidated semi-annual revenue'].on_click(lambda x: display(consolidated.revenue_analysis(data_dic,input_dic)))
items['Consolidated annual revenue - FY'].on_click(lambda x: display(consolidated.annual_revenue_analysis(data_dic,input_dic)))
items['Consolidated annual revenue - CY'].on_click(lambda x: display(consolidated.annual_revenue_analysis(data_dic,input_dic,cal_year=True)))
widgets.HBox([left_box,right_box])



HBox(children=(VBox(children=(Button(description='Semi-annual revenue', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen')), Button(description='Annual revenue - FY', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen')), Button(description='Annual revenue - CY', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen'))), layout=Layout(align_items='stretch', border='solid', display='flex', flex_flow='column', width='30%')), VBox(children=(Button(description='Consolidated semi-annual revenue', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen')), Button(description='Consolidated annual revenue - FY', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen')), Button(description='Consolidated annual revenue - CY', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen'))), layout=Layout(align_items='stretch', border='solid', display='flex', flex_flow='column', width='30%'))))

In [6]:
costs_types = ['Semi-annual costs','Annual costs - FY','Annual costs - CY','Consolidated semi-annual costs','Consolidated annual costs - FY','Consolidated annual costs - CY']
items2 = {}
for w2 in costs_types:
    items2[w2] = widgets.Button(description=w2,layout=widgets.Layout(width='100%', height='40px'))
    items2[w2].style.button_color = 'lightgreen'
left_box2 = widgets.VBox([list(items2.values())[0],list(items2.values())[1],list(items2.values())[2]],layout=box_layout)
right_box2 = widgets.VBox([list(items2.values())[3],list(items2.values())[4],list(items2.values())[5]],layout=box_layout)
items2['Semi-annual costs'].on_click(lambda x: display(costs.semi_costs(input_dic)))
items2['Annual costs - FY'].on_click(lambda x: display(costs.annual_costs(input_dic)))
items2['Annual costs - CY'].on_click(lambda x: display(costs.annual_costs(input_dic,cal_year=True)))
items2['Consolidated semi-annual costs'].on_click(lambda x: display(consolidated.costs_analysis(input_dic)))
items2['Consolidated annual costs - FY'].on_click(lambda x: display(consolidated.annual_costs_analysis(input_dic)))
items2['Consolidated annual costs - CY'].on_click(lambda x: display(consolidated.annual_costs_analysis(input_dic,cal_year=True)))
widgets.HBox([left_box2,right_box2])

HBox(children=(VBox(children=(Button(description='Semi-annual costs', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen')), Button(description='Annual costs - FY', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen')), Button(description='Annual costs - CY', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen'))), layout=Layout(align_items='stretch', border='solid', display='flex', flex_flow='column', width='30%')), VBox(children=(Button(description='Consolidated semi-annual costs', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen')), Button(description='Consolidated annual costs - FY', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen')), Button(description='Consolidated annual costs - CY', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen'))), layout=Layout(align_items='stretch', border='solid', display='flex', flex_flow='column', width='30%'))))

In [7]:
ops = [i for i in list(range(general.recent_end_year, general.recent_end_year+10))]
a_slider = widgets.SelectMultiple(options=ops, description='Years for comparison')
display(a_slider)

compare1 = ['NNB comparison','AUA comparison']
box_layout2 = widgets.Layout(display='flex', flex_flow='row', align_items='stretch',border='solid',width='30%')
items3 = {}
for w3 in compare1:
    items3[w3] = widgets.Button(description=w3,layout=widgets.Layout(width='100%', height='40px'))
    items3[w3].style.button_color = 'lightgreen'
box3 = widgets.Box([list(items3.values())[0],list(items3.values())[1]],layout=box_layout2)
items3['NNB comparison'].on_click(lambda x: display(consolidated.get_nnb_compare(data_dic, input_dic,list(a_slider.value))))
items3['AUA comparison'].on_click(lambda x: display(consolidated.get_aua_compare(data_dic, input_dic,list(a_slider.value))))


display(box3)

SelectMultiple(description='Years for comparison', options=(2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025, 2026, 2027), value=())

Box(children=(Button(description='NNB comparison', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen')), Button(description='AUA comparison', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen'))), layout=Layout(align_items='stretch', border='solid', display='flex', flex_flow='row', width='30%'))

In [8]:
compare2 = ['Interim revenue comparison','Annual revenue comparison', 'Interim costs comparison', 'Annual costs comparison']
box_layout3 = widgets.Layout(display='flex', flex_flow='row', align_items='stretch',border='solid',width='50%')
items4 = {}
for w4 in compare2:
    items4[w4] = widgets.Button(description=w4,layout=widgets.Layout(width='100%', height='40px'))
    items4[w4].style.button_color = 'lightgreen'
top_box4 = widgets.HBox([list(items4.values())[0],list(items4.values())[1]],layout=box_layout3)
bottom_box4 = widgets.HBox([list(items4.values())[2],list(items4.values())[3]],layout=box_layout3)
items4['Interim revenue comparison'].on_click(lambda x: display(consolidated.get_revenue_compare(data_dic, input_dic,True,list(a_slider.value))))
items4['Annual revenue comparison'].on_click(lambda x: display(consolidated.get_revenue_compare(data_dic, input_dic,False,list(a_slider.value))))
items4['Interim costs comparison'].on_click(lambda x: display(consolidated.get_costs_compare(input_dic,True,list(a_slider.value))))
items4['Annual costs comparison'].on_click(lambda x: display(consolidated.get_costs_compare(input_dic,False,list(a_slider.value))))

widgets.VBox([top_box4, bottom_box4])

VBox(children=(HBox(children=(Button(description='Interim revenue comparison', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen')), Button(description='Annual revenue comparison', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen'))), layout=Layout(align_items='stretch', border='solid', display='flex', flex_flow='row', width='50%')), HBox(children=(Button(description='Interim costs comparison', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen')), Button(description='Annual costs comparison', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen'))), layout=Layout(align_items='stretch', border='solid', display='flex', flex_flow='row', width='50%'))))

In [9]:
'''=====current values=======
discounted cash flow period: 9 years
perpetuity growth rate: 0.035
cash flow discounted rate: 0.085
'''
dcf_period = widgets.IntText(value=9,placeholder='Integer',description='DCF period - No. of years:')
display(dcf_period)
perp_growth = widgets.FloatText(value=0.035,placeholder='Percentage as decimals',description='Perpetuity growth rate:')
display(perp_growth)
disct_rate = widgets.FloatText(value=0.085,placeholder='Percentage as decimals',description='Cash flow discount rate:')
display(disct_rate)
current_value = widgets.Checkbox(value=False, description='Fair value now')
display(current_value)
fractional = widgets.Checkbox(value=False, description='Fractional valuation')
display(fractional)

dis_cash = ['Discounted cash flow','Fair value']
items5 = {}
for w5 in dis_cash:
    items5[w5] = widgets.Button(description=w5, layout=widgets.Layout(width='100%', height='40px'))
    items5[w5].style.button_color = 'lightgreen'
fair = widgets.HBox([list(items5.values())[0],list(items5.values())[1]],layout=box_layout3)
items5['Discounted cash flow'].on_click(lambda x: display(discf.disc_cash_flow(data_dic, input_dic,now=current_value.value,fractional=fractional.value,dcf_p=dcf_period.value,disc_rate=disct_rate.value)))
items5['Fair value'].on_click(lambda x: display(discf.fair_value(data_dic, input_dic,now=current_value.value,fractional=fractional.value,dcf_p=dcf_period.value,disc_rate=disct_rate.value,pep_rate=perp_growth.value)))    
display(fair)

IntText(value=9, description='DCF period - No. of years:')

FloatText(value=0.035, description='Perpetuity growth rate:')

FloatText(value=0.085, description='Cash flow discount rate:')

Checkbox(value=False, description='Fair value now')

Checkbox(value=False, description='Fractional valuation')

HBox(children=(Button(description='Discounted cash flow', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen')), Button(description='Fair value', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen'))), layout=Layout(align_items='stretch', border='solid', display='flex', flex_flow='row', width='50%'))

In [10]:
radio1 = widgets.RadioButtons(options=['month_no','quarter_no','half_no','financial_year','calendar_year'],description='Period',disabled=False)
display(radio1)

others = ['HLF composite monthly return','Total NNB hlf Algo','NNB distribution hlf Algo','Total AUA','% Revenue distribution','Avg AUA distribution', 'Total NNB nnc Algo', 'NNB distribution nnc Algo']
items6 = {}
for w6 in others:
    items6[w6] = widgets.Button(description=w6, layout=widgets.Layout(width='100%', height='40px'))
    items6[w6].style.button_color = 'lightgreen'
other_box1 = widgets.VBox([list(items6.values())[0],list(items6.values())[1],list(items6.values())[2],list(items6.values())[3]],layout=box_layout)
other_box2 = widgets.VBox([list(items6.values())[4],list(items6.values())[5],list(items6.values())[6],list(items6.values())[7]],layout=box_layout)
items6['HLF composite monthly return'].on_click(lambda x: display(discretionary_aua.get_composite_return(data_dic)))
items6['Total NNB hlf Algo'].on_click(lambda x: display(combined.total_nnb(data_dic, input_dic, opt=radio1.value)))
items6['NNB distribution hlf Algo'].on_click(lambda x: display(combined.nnb_distribution(data_dic, input_dic, opt=radio1.value)))
items6['Total AUA'].on_click(lambda x: display(combined.total_aua(data_dic, input_dic)))
items6['% Revenue distribution'].on_click(lambda x: display(stats.summary_revenue_dist_percent(data_dic, input_dic)))
items6['Avg AUA distribution'].on_click(lambda x: display(stats.summary_avg_aua_dist(data_dic, input_dic,radio1.value)))
items6['Total NNB nnc Algo'].on_click(lambda x: display(combined.total_nnb_ci_clientAlgo(data_dic, input_dic, opt=radio1.value)))
items6['NNB distribution nnc Algo'].on_click(lambda x: display(combined.total_nnb_distribution_clientAlgo(data_dic, input_dic, opt=radio1.value)))

widgets.HBox([other_box1, other_box2])

#Financial year based

RadioButtons(description='Period', options=('month_no', 'quarter_no', 'half_no', 'financial_year', 'calendar_year'), value='month_no')

HBox(children=(VBox(children=(Button(description='HLF composite monthly return', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen')), Button(description='Total NNB hlf Algo', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen')), Button(description='NNB distribution hlf Algo', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen')), Button(description='Total AUA', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen'))), layout=Layout(align_items='stretch', border='solid', display='flex', flex_flow='column', width='30%')), VBox(children=(Button(description='% Revenue distribution', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen')), Button(description='Avg AUA distribution', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen')), Button(description='Total NNB nnc Algo', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen')), Button(description='NNB distribution nnc Algo', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen'))), layout=Layout(align_items='stretch', border='solid', display='flex', flex_flow='column', width='30%'))))

,,,,,,cash_service_aua,pms_hlf_aua,pms_others_aua,thirdparty_hlf_aua,vantage_cash_aua,vantage_hl_shares_aua,vantage_hlf_aua,vantage_other_funds_aua,vantage_other_shares_aua,hlf_aua,...,total_hlf_aua,total_funds_aua,total_assets_aua,Funds,Shares,HLF,Cash,SIPP,ISA,F&S
month_end,financial_year,quarter_no,half_no,calendar_year,month_no,,,,,,,,,,,,,,,,,,,,,
2018-06-29,2018,4,2,2018,6,"20,000,000.000000","3,500,000,000.000000","100,000,000.000000","112,314,814.814815","9,600,000,000.000000","3,700,000,000.000000","5,952,685,185.185185","41,315,000,000.000000","27,300,000,000.000000","6,065,000,000.000000",...,"9,565,000,000.000000","50,980,000,000.000000","91,600,000,000.000000","50,980,000,000.000000","31,000,000,000.000000","9,565,000,000.000000","9,600,000,000.000000","26,394,000,000.000000","36,071,800,000.000000","25,514,200,000.000000"
2018-07-31,2019,1,1,2018,7,"22,911,227.844311","3,527,843,784.940634","101,262,770.965624","113,757,019.245857","9,668,599,235.976461","3,896,119,705.807761","6,041,140,840.309797","42,494,280,384.028793","27,839,644,773.943802","6,154,897,859.555655",...,"9,682,741,644.496288","52,278,284,799.490707","93,705,559,743.063034","52,278,284,799.490707","31,735,764,479.751564","9,682,741,644.496288","9,668,599,235.976461","27,016,062,587.793739","36,921,952,203.318108","26,115,527,168.200615"
2018-08-31,2019,1,1,2018,8,"25,822,455.688623","3,514,699,834.487024","102,525,541.931248","113,883,929.416761","9,737,198,471.952921","4,128,835,911.742329","6,059,885,899.647093","42,094,874,068.407372","27,177,412,198.407234","6,173,769,829.063854",...,"9,688,469,663.550879","51,885,869,273.889496","92,955,138,311.680618","51,885,869,273.889496","31,306,248,110.149563","9,688,469,663.550879","9,737,198,471.952921","26,793,627,143.872112","36,617,957,096.625221","25,900,506,239.076374"
2018-09-28,2019,1,1,2018,9,"29,683,673.281269","3,516,452,160.905263","104,200,632.335069","114,668,021.237804","9,828,182,960.615309","4,128,835,911.742329","6,117,385,966.523550","42,448,421,770.637772","27,438,647,976.468090","6,232,053,987.761353",...,"9,748,506,148.666616","52,301,128,551.639458","93,726,479,073.746445","52,301,128,551.639458","31,567,483,888.210419","9,748,506,148.666616","9,828,182,960.615309","27,022,842,782.167454","36,931,218,468.962189","26,122,081,356.095207"
2018-10-31,2019,2,1,2018,10,"32,936,926.831731","3,552,113,243.062370","106,502,982.603377","116,388,761.176938","9,993,926,942.668972","4,169,128,371.604680","6,221,210,287.287728","43,134,055,865.935555","27,907,588,449.481747","6,337,599,048.464666",...,"9,889,712,291.527037","53,130,271,140.065964","95,233,851,830.653076","53,130,271,140.065964","32,076,716,821.086426","9,889,712,291.527037","9,993,926,942.668972","27,462,689,603.446682","37,532,342,458.043800","26,547,266,616.665123"
2018-11-30,2019,2,1,2018,11,"36,088,704.032975","3,570,760,733.214783","108,318,961.483653","117,562,668.039556","10,113,062,571.308411","4,189,421,814.766872","6,296,033,446.311190","43,617,954,700.030884","28,246,494,829.667545","6,413,596,114.350745",...,"9,984,356,847.565529","53,710,630,509.080063","96,295,698,428.855865","53,710,630,509.080063","32,435,916,644.434418","9,984,356,847.565529","10,113,062,571.308411","27,774,159,009.037334","37,958,017,312.351021","26,848,353,708.736088"
2018-12-31,2019,2,1,2018,12,"39,240,982.447306","3,589,492,257.085817","110,137,342.654777","118,739,275.826051","10,232,428,408.433807","4,209,814,037.289801","6,370,999,848.188054","44,102,849,947.121834","28,586,043,475.658516","6,489,739,124.014105",...,"10,079,231,381.099922","54,292,218,670.876534","97,359,745,574.705978","54,292,218,670.876534","32,795,857,512.948318","10,079,231,381.099922","10,232,428,408.433807","28,086,262,497.755421","38,384,558,746.932404","27,150,053,747.830238"
2019-01-31,2019,3,2,2019,1,"122,762,683.695500","3,608,475,624.072889","112,118,159.785443","119,993,502.250744","10,360,718,667.447329","4,230,305,519.987000","6,451,603,191.772152","4

In [11]:
nnb_radio = widgets.RadioButtons(options=['day','month','semi-annual','quarter','annual'],description='Period',disabled=False)
nnb_dropdown = widgets.Dropdown(options=['no_select',None],description='Fund option',disabled=False)
unit_dropdown = widgets.Dropdown(options=['acc','inc'],description='Unit type',disabled=False)
display(nnb_radio)
display(nnb_dropdown)
display(unit_dropdown)

nnb_b = widgets.Button(description='Implied HLF nnb period-to-date', layout=widgets.Layout(width='50%', height='40px'))
nnb_b.style.button_color = 'lightgreen'

nnb_b2 = widgets.Button(description='HLF unit change period-to-date', layout=widgets.Layout(width='50%', height='40px'))
nnb_b2.style.button_color = 'lightgreen'
nnb_b.on_click(lambda x: display(stats.hlf_to_date_implied_nnb(data_dic,typ=nnb_radio.value,fund_opt=nnb_dropdown.value)))
nnb_b2.on_click(lambda x: display(stats.hlf_to_date_unit_change(data_dic, unit_type=unit_dropdown.value,typ=nnb_radio.value,fund_opt=nnb_dropdown.value)))

display(nnb_b)
display(nnb_b2)

RadioButtons(description='Period', options=('day', 'month', 'semi-annual', 'quarter', 'annual'), value='day')

Dropdown(description='Fund option', options=('no_select', None), value='no_select')

Dropdown(description='Unit type', options=('acc', 'inc'), value='acc')

Button(description='Implied HLF nnb period-to-date', layout=Layout(height='40px', width='50%'), style=ButtonStyle(button_color='lightgreen'))

Button(description='HLF unit change period-to-date', layout=Layout(height='40px', width='50%'), style=ButtonStyle(button_color='lightgreen'))

In [12]:
cash_radio = widgets.RadioButtons(options=['monthly','quarter_no','half_no','financial_year','calendar_year'],description='Period',disabled=False)
display(cash_radio)


cash_b = widgets.Button(description='Cash margin', layout=widgets.Layout(width='50%', height='40px'))
cash_b.style.button_color = 'lightgreen'
cash_b.on_click(lambda x: display(stats.cash_margin(data_dic, input_dic, cash_radio.value)))
display(cash_b)

RadioButtons(description='Period', options=('monthly', 'quarter_no', 'half_no', 'financial_year', 'calendar_year'), value='monthly')

Button(description='Cash margin', layout=Layout(height='40px', width='50%'), style=ButtonStyle(button_color='lightgreen'))

In [13]:
eps_radio = widgets.RadioButtons(options=[True, False],description='Calendar year?',disabled=False)
display(eps_radio)

eps_summary = widgets.Button(description='Net earnings and EPS', layout=widgets.Layout(width='50%', height='40px'))
eps_summary.style.button_color = 'lightgreen'
eps_summary.on_click(lambda x: display(stats.summary_total(data_dic, input_dic, cal=eps_radio.value)))
display(eps_summary)


RadioButtons(description='Calendar year?', options=(True, False), value=True)

Button(description='Net earnings and EPS', layout=Layout(height='40px', width='50%'), style=ButtonStyle(button_color='lightgreen'))

In [14]:
hlf_radio = widgets.RadioButtons(options=['month_no','quarter_no','half_no','financial_year','calendar_year'],description='Period',disabled=False)
display(hlf_radio)


buttom3 = ['HLF revenue margin','Monthly revenue', 'HLF avg fund size','Quarter Trading update revenue']
items7 = {}
for w7 in buttom3:
    items7[w7] = widgets.Button(description=w7, layout=widgets.Layout(width='100%', height='40px'))
    items7[w7].style.button_color = 'lightgreen'

top_m = widgets.HBox([list(items7.values())[0],list(items7.values())[1]],layout=box_layout3)
bottom_m = widgets.HBox([list(items7.values())[2],list(items7.values())[3]],layout=box_layout3)

items7['HLF revenue margin'].on_click(lambda x: display(stats.hlf_revenue_margin(data_dic,input_dic,hlf_radio.value)))
items7['Monthly revenue'].on_click(lambda x: display(revenue.monthly_revenue(data_dic, input_dic)))
items7['HLF avg fund size'].on_click(lambda x: display(stats.avg_hlf_size(data_dic, input_dic,hlf_radio.value)))
items7['Quarter Trading update revenue'].on_click(lambda x: display(stats.quarter_revenue(data_dic, input_dic)))


widgets.VBox([top_m, bottom_m])

RadioButtons(description='Period', options=('month_no', 'quarter_no', 'half_no', 'financial_year', 'calendar_year'), value='month_no')

VBox(children=(HBox(children=(Button(description='HLF revenue margin', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen')), Button(description='Monthly revenue', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen'))), layout=Layout(align_items='stretch', border='solid', display='flex', flex_flow='row', width='50%')), HBox(children=(Button(description='HLF avg fund size', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen')), Button(description='Quarter Trading update revenue', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen'))), layout=Layout(align_items='stretch', border='solid', display='flex', flex_flow='row', width='50%'))))

In [15]:
display(a_slider)

client_options = ['Total No. of clients','Net new clients']
items8 = {}
for w8 in client_options:
    items8[w8] = widgets.Button(description=w8, layout=widgets.Layout(width='100%', height='40px'))
    items8[w8].style.button_color = 'lightgreen'
c_fair = widgets.HBox([list(items8.values())[0],list(items8.values())[1]],layout=box_layout3)
items8['Total No. of clients'].on_click(lambda x: display(combined.total_client_predt(data_dic, input_dic).loc[list(a_slider.value)]))
items8['Net new clients'].on_click(lambda x: display(combined.net_new_client_predt(data_dic, input_dic).loc[list(a_slider.value)]))
display(c_fair)

SelectMultiple(description='Years for comparison', options=(2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025, 2026, 2027), value=())

HBox(children=(Button(description='Total No. of clients', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen')), Button(description='Net new clients', layout=Layout(height='40px', width='100%'), style=ButtonStyle(button_color='lightgreen'))), layout=Layout(align_items='stretch', border='solid', display='flex', flex_flow='row', width='50%'))

In [16]:
# %load_ext autoreload
# %autoreload 2

In [17]:
'''
fair value sensitivity analysis

disc_rate_list = [0.0825,0.085,0.0875,0.09]
pep_rate_list = [0.035,0.0375,0.04,0.0425]

df_now = pandas.DataFrame(columns = disc_rate_list, index=pep_rate_list)
df_now.columns.name = 'Cost of Equity'
df_now.index.name = 'Long-term Growth Rate'

for i in pep_rate_list:
    for j in disc_rate_list:
        df_now.loc[i,j] = discf.fair_value(data_dic, input_dic,now=True,dcf_p=9,disc_rate=j,pep_rate=i).loc['Fair value per share','HL']
df_1y = pandas.DataFrame(columns = disc_rate_list, index=pep_rate_list)
df_1y.columns.name = 'Cost of Equity'
df_1y.index.name = 'Long-term Growth Rate'

for i in pep_rate_list:
    for j in disc_rate_list:
        df_1y.loc[i,j] = discf.fair_value(data_dic, input_dic,now=False,dcf_p=9,disc_rate=j,pep_rate=i).loc['Fair value per share','HL']
'''

"\nfair value sensitivity analysis\n\ndisc_rate_list = [0.0825,0.085,0.0875,0.09]\npep_rate_list = [0.035,0.0375,0.04,0.0425]\n\ndf_now = pandas.DataFrame(columns = disc_rate_list, index=pep_rate_list)\ndf_now.columns.name = 'Cost of Equity'\ndf_now.index.name = 'Long-term Growth Rate'\n\nfor i in pep_rate_list:\n    for j in disc_rate_list:\n        df_now.loc[i,j] = discf.fair_value(data_dic, input_dic,now=True,dcf_p=9,disc_rate=j,pep_rate=i).loc['Fair value per share','HL']\ndf_1y = pandas.DataFrame(columns = disc_rate_list, index=pep_rate_list)\ndf_1y.columns.name = 'Cost of Equity'\ndf_1y.index.name = 'Long-term Growth Rate'\n\nfor i in pep_rate_list:\n    for j in disc_rate_list:\n        df_1y.loc[i,j] = discf.fair_value(data_dic, input_dic,now=False,dcf_p=9,disc_rate=j,pep_rate=i).loc['Fair value per share','HL']\n"